In [1]:
from pomegranate import *

# creiamo il nodo pioggia senza genitori
pioggia = Node(DiscreteDistribution({
    "no": 0.7,
    "lieve": 0.2,
    "intensa": 0.1
}), name="pioggia")

# il nodo della manutenzione sulla linea ferroviaria dipende dalla pioggia
manutenzione = Node(ConditionalProbabilityTable([
    ["no", "si", 0.4],
    ["no", "no", 0.6],
    ["lieve", "si", 0.2],
    ["lieve", "no", 0.8],
    ["intensa", "si", 0.1],
    ["intensa", "no", 0.9]
], [pioggia.distribution]), name="manutenzione")

# il nodo del treno è condizionato alla pioggia e alla manutenzione
treno = Node(ConditionalProbabilityTable([
    ["no", "si", "puntuale", 0.8],
    ["no", "si", "ritardo", 0.2],
    ["no", "no", "puntuale", 0.9],
    ["no", "no", "ritardo", 0.1],
    ["lieve", "si", "puntuale", 0.6],
    ["lieve", "si", "ritardo", 0.4],
    ["lieve", "no", "puntuale", 0.7],
    ["lieve", "no", "ritardo", 0.3],
    ["intensa", "si", "puntuale", 0.4],
    ["intensa", "si", "ritardo", 0.6],
    ["intensa", "no", "puntuale", 0.5],
    ["intensa", "no", "ritardo", 0.5],
], [pioggia.distribution, manutenzione.distribution]), name="treno")

# il nodo dell'appuntamento è condizionato al treno
appuntamento = Node(ConditionalProbabilityTable([
    ["puntuale", "presente", 0.9],
    ["puntuale", "assente", 0.1],
    ["ritardo", "presente", 0.6],
    ["ritardo", "assente", 0.4]
], [treno.distribution]), name="appuntamento")
# Possamo creare il modellolo di rete di Bayes inserendo tutti i nodi e specificando chi è genitore di chi
# 
modello = BayesianNetwork()
modello.add_states(pioggia, manutenzione, treno, appuntamento)

# aggiungiamo la descrizione delle dipendenze con gli archi (edge) da un nodo all'altro
modello.add_edge(pioggia, manutenzione)
modello.add_edge(pioggia, treno)
modello.add_edge(manutenzione, treno)
modello.add_edge(treno, appuntamento)

# usiamo/attiviamo il modello!
modello.bake()


Qual è la probabilità di nessuna pioggia e nessun lavoro sulla linea e treno in orario e sono presente all’evento/appuntamento?
Per interrogare il modello usiamo la conoscenza che abbiamo per calcolare la probabilità cercata:


In [2]:

probabilita = modello.probability([["no", "no", "puntuale", "presente"]])

print(probabilita)

0.34019999999999995


Possiamo anche usare il modello per fare delle previsioni basandoci su qualche elemento di conoscenza. 
Ad esempio se sappiamo che il treno è in ritardo cosa possiamo dire delle probabilità dei vari eventi (valori del dominio delle variabili aleatorie modellate)?

In [3]:

predizioni = modello.predict_proba({
    "treno": "ritardo"
})

# Print predizione for each nodo
for nodo, predizione in zip(modello.states, predizioni):
    if isinstance(predizione, str):
        print(f"{nodo.name}: {predizione}")
    else:
        print(f"{nodo.name}")
        for value, probabilita in predizione.parameters[0].items():
            print(f"    {value}: {probabilita:.4f}")

pioggia
    no: 0.4583
    lieve: 0.3069
    intensa: 0.2348
manutenzione
    no: 0.6432
    si: 0.3568
treno: ritardo
appuntamento
    presente: 0.6000
    assente: 0.4000


Vedete? Data la conoscenza della circostanza che il treno è in ritardo il modello mi consente di inferire altre informazioni probabilistiche!
Grandioso, vero?
